In [ ]:
from google.colab import files
uploaded =files.upload()

Saving Online retail.xlsx to Online retail.xlsx


In [ ]:
import pandas as pd

# Load the dataset
file_path = "/content/Online retail.xlsx"
df = pd.read_excel(file_path)

# Display basic information and first few rows
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                           --------------  ----- 
 0   shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil  7500 non-null   object
dtypes: object(1)
memory usage: 58.7+ KB


(None,
   shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
 0                             burgers,meatballs,eggs                                                                                                                                                                             
 1                                            chutney                                                                                                                                                                             
 2                                     turkey,avocado                                                                                                                                                                             
 3  mineral water,milk,energy bar,whole wheat rice...                                

In [ ]:
# Convert the single column into a list of transactions
df_cleaned = df.iloc[:, 0].dropna().apply(lambda x: x.split(','))

# Convert to a list of lists format required for Apriori algorithm
transactions = df_cleaned.tolist()

# Display first few transactions
transactions[:5]


[['burgers', 'meatballs', 'eggs'],
 ['chutney'],
 ['turkey', 'avocado'],
 ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea'],
 ['low fat yogurt']]

In [ ]:
from itertools import combinations
from collections import defaultdict

# Function to calculate support
def get_support(itemset, transactions):
    count = sum(1 for transaction in transactions if itemset.issubset(transaction))
    return count / len(transactions)

# Generate frequent itemsets
min_support = 0.01  # Minimum support threshold
item_counts = defaultdict(int)

# Count occurrences of individual items
for transaction in transactions:
    for item in transaction:
        item_counts[frozenset([item])] += 1

# Convert to support values
num_transactions = len(transactions)
frequent_items = {itemset: count / num_transactions for itemset, count in item_counts.items() if count / num_transactions >= min_support}

# Generate frequent item pairs
pair_counts = defaultdict(int)
for transaction in transactions:
    for itemset in combinations(transaction, 2):
        pair_counts[frozenset(itemset)] += 1

# Convert to support values
frequent_pairs = {itemset: count / num_transactions for itemset, count in pair_counts.items() if count / num_transactions >= min_support}

# Display first few frequent pairs
list(frequent_pairs.items())[:5]


[(frozenset({'burgers', 'eggs'}), 0.0288),
 (frozenset({'milk', 'mineral water'}), 0.048),
 (frozenset({'mineral water', 'whole wheat rice'}), 0.020133333333333333),
 (frozenset({'green tea', 'mineral water'}), 0.030933333333333334),
 (frozenset({'milk', 'whole wheat rice'}), 0.011866666666666666)]

In [ ]:
[(frozenset({'milk', 'bread'}), 0.05),
 (frozenset({'eggs', 'bread'}), 0.04),
 (frozenset({'milk', 'butter'}), 0.03)]


[(frozenset({'bread', 'milk'}), 0.05),
 (frozenset({'bread', 'eggs'}), 0.04),
 (frozenset({'butter', 'milk'}), 0.03)]

In [ ]:
# Function to calculate confidence
def get_confidence(antecedent, consequent, transactions):
    antecedent_support = get_support(antecedent, transactions)
    combined_support = get_support(antecedent | consequent, transactions)
    return combined_support / antecedent_support if antecedent_support > 0 else 0

# Function to calculate lift
def get_lift(antecedent, consequent, transactions):
    confidence = get_confidence(antecedent, consequent, transactions)
    consequent_support = get_support(consequent, transactions)
    return confidence / consequent_support if consequent_support > 0 else 0

# Generate association rules
min_confidence = 0.2
rules = []

for itemset in frequent_pairs.keys():
    for item in itemset:
        antecedent = frozenset([item])
        consequent = itemset - antecedent

        confidence = get_confidence(antecedent, consequent, transactions)
        lift = get_lift(antecedent, consequent, transactions)

        if confidence >= min_confidence and lift > 1:
            rules.append((antecedent, consequent, confidence, lift))

# Convert to DataFrame for better visualization
rules_df = pd.DataFrame(rules, columns=["Antecedent", "Consequent", "Confidence", "Lift"])

# Display top association rules
rules_df.sort_values(by="Lift", ascending=False).head()


,Antecedent,Consequent,Confidence,Lift
77,(herb & pepper),(ground beef),0.323450,3.291555
64,(tomatoes),(frozen vegetables),0.235867,2.474134
101,(shrimp),(frozen vegetables),0.233645,2.450820
35,(soup),(milk),0.300792,2.320923
37,(ground beef),(spaghetti),0.398915,2.290857


### 1. **What is Lift and Why is it Important in Association Rules?**  
**Lift** measures how much more likely item **Y** is bought when item **X** is bought, compared to random chance. It helps assess the **strength** of an association rule.  
- **Formula**:  
  \[
  \text{Lift}(X \Rightarrow Y) = \frac{\text{Confidence}(X \Rightarrow Y)}{\text{Support}(Y)}
  \]  
- **Importance**: A **lift > 1** means a strong positive correlation, **=1** means no association, and **<1** means a negative relationship.

### 2. **What are Support and Confidence? How Do You Calculate Them?**  
- **Support**: Measures how frequently an itemset appears in transactions.  
  \[
  \text{Support}(X) = \frac{\text{Transactions containing X}}{\text{Total transactions}}
  \]  
- **Confidence**: Measures the likelihood of item **Y** appearing when item **X** is present.  
  \[
  \text{Confidence}(X \Rightarrow Y) = \frac{\text{Support}(X \cap Y)}{\text{Support}(X)}
  \]

### 3. **Limitations or Challenges of Association Rule Mining**  
- **High Computational Cost**: Large datasets generate many rules, making it expensive.  
- **Too Many Rules**: Can result in excessive, low-quality, or irrelevant rules.  
- **Low Support for Rare but Valuable Patterns**: Rare but important patterns may be missed.  
- **Interpretability Issues**: Not all rules are actionable or meaningful.  

